# Dự Đoán Giá Cổ Phiếu FPT - LTSF-Linear với Advanced Grid Search

## Mục tiêu:
- Dự đoán giá đóng cửa FPT cho 100 ngày tiếp theo
- Grid Search toàn diện trên:
  - **Models**: Linear, DLinear, NLinear
  - **Variants**: Univariate (1 feature) vs Multivariate (nhiều features)
  - **Normalization**: RevIN vs No RevIN
  - **Sequence Lengths**: 7, 15, 30, 60, 120, 480
  - **Regime Switching**: HMM với 3-4 regimes

## 1. Import Libraries và Setup

In [ ]:
# Install dependencies if needed
!pip install hmmlearn -q

In [ ]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from datetime import timedelta
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hmmlearn.hmm import GaussianHMM

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

def seed_everything(seed=42):
    """Set random seed for reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

## 2. Configuration - Hyperparameters

In [ ]:
# === CONFIGURATION ===
DATA_PATH = 'data/FPT_train.csv'
SUBMISSION_DIR = 'submissions/default'
PRED_LEN = 100  # Predict 100 days ahead
TARGET_COL = 'close'

# Grid Search Space
MODEL_TYPES = ['GLinear']
VARIANTS = ['Univariate', 'Multivariate']
# USE_REVIN_OPTIONS removed - GLinear always uses RevIN
SEQ_LENS = [7, 15, 30, 60, 120, 480]
USE_HMM_OPTIONS = [False, True]
N_REGIMES_OPTIONS = [3, 4]
REGIME_WINDOWS = [30, 60]

# Training Hyperparams
BATCH_SIZE = 32
EPOCHS = 1000
PATIENCE = 15
LEARNING_RATE = 1e-3

# Create directories
os.makedirs(SUBMISSION_DIR, exist_ok=True)
os.makedirs('results', exist_ok=True)

print(f"Configuration:")
print(f"  - Prediction Length: {PRED_LEN} days")
print(f"  - Model Types: {MODEL_TYPES}")
print(f"  - Variants: {VARIANTS}")
print(f"  - Sequence Lengths: {SEQ_LENS}")
print(f"  - HMM Regimes: {N_REGIMES_OPTIONS}")

# Calculate total experiments

# 1. Count No-HMM experiments
# Variants * Seq_Lens * Models
total_no_hmm = len(VARIANTS) * len(SEQ_LENS) * len(MODEL_TYPES)

# 2. Count HMM experiments
# HMM only runs on seq_len < 120
short_seq_lens_count = len([s for s in SEQ_LENS if s < 120])
# Variants * Short_Seq_Lens * Models * HMM_Configs
hmm_configs_count = len(N_REGIMES_OPTIONS) * len(REGIME_WINDOWS)
total_hmm = len(VARIANTS) * short_seq_lens_count * len(MODEL_TYPES) * hmm_configs_count

total_experiments = total_no_hmm + total_hmm

print(f"\nCalculation Logic:")
print(f"  - No HMM Experiments: {len(VARIANTS)} * {len(SEQ_LENS)} * {len(MODEL_TYPES)} = {total_no_hmm}")
print(f"  - HMM Experiments: {len(VARIANTS)} * {short_seq_lens_count} * {len(MODEL_TYPES)} * {hmm_configs_count} = {total_hmm}")
print(f"\nEstimated total experiments: {total_experiments}")

## 3. Load Data và EDA

In [ ]:
# Load data
df = pd.read_csv(DATA_PATH)
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['time'].min().date()} to {df['time'].max().date()}")
print(f"\nFirst few rows:")
display(df.head())

print(f"\nData types:")
print(df.dtypes)

print(f"\nBasic statistics:")
display(df.describe())

In [ ]:
# Visualize price history
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('FPT Stock Price - EDA', fontsize=16, fontweight='bold')

# Price history
axes[0, 0].plot(df['time'], df['close'], color='blue', linewidth=1)
axes[0, 0].set_title('Close Price History')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price')
axes[0, 0].grid(True, alpha=0.3)

# Volume
axes[0, 1].bar(df['time'], df['volume'], color='green', alpha=0.5, width=1)
axes[0, 1].set_title('Volume')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Volume')
axes[0, 1].grid(True, alpha=0.3)

# Daily returns distribution
returns = df['close'].pct_change().dropna()
axes[1, 0].hist(returns, bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Daily Returns Distribution')
axes[1, 0].set_xlabel('Return')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].axvline(x=0, color='red', linestyle='--')
axes[1, 0].grid(True, alpha=0.3)

# Rolling volatility
rolling_vol = returns.rolling(window=30).std() * np.sqrt(252)
axes[1, 1].plot(df['time'].iloc[1:], rolling_vol, color='red', linewidth=1)
axes[1, 1].set_title('30-day Rolling Volatility (Annualized)')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Volatility')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Feature Engineering & Data Preprocessing

In [ ]:
# === FEATURE ENGINEERING ===

# Log Transform (stabilize variance)
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[f'{col}_log'] = np.log1p(df[col])

# Spread Features
df['HL_Spread'] = df['high_log'] - df['low_log']  # Intraday range
df['OC_Spread'] = df['close_log'] - df['open_log']  # Open-Close spread

# Returns & Volatility for HMM Regime Detection
df['returns'] = df['close'].pct_change().fillna(0)
df['volatility'] = df['returns'].rolling(window=10).std().fillna(0)
df['trend'] = df['close'].rolling(window=10).mean().pct_change().fillna(0)

print("Features created:")
print(f"  - Log transforms: open_log, high_log, low_log, close_log, volume_log")
print(f"  - Spread features: HL_Spread, OC_Spread")
print(f"  - HMM features: returns, volatility, trend")

print(f"\nDataset shape after feature engineering: {df.shape}")
display(df.tail())

In [ ]:
# Check for missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    print("Missing values:")
    print(missing[missing > 0])
else:
    print("✓ No missing values!")

## 5. Dataset và Sliding Window

In [ ]:
class TimeSeriesDataset(Dataset):
    """PyTorch Dataset for time series forecasting"""
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def create_sliding_window(data, seq_len, pred_len, target_col_idx, feature_cols_idx):
    """
    Create sliding window sequences for time series forecasting.
    """
    X, y = [], []
    for i in range(len(data) - seq_len - pred_len + 1):
        X.append(data[i : i + seq_len, feature_cols_idx])
        y.append(data[i + seq_len : i + seq_len + pred_len, target_col_idx])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

print("Dataset classes defined!")

## 6. Regime Detector (Hidden Markov Model)

In [ ]:
class RegimeDetector:
    """
    Detect market regimes using Gaussian HMM.
    Uses returns, volatility, and trend as features.
    """
    def __init__(self, n_components=3, window=30):
        self.n_components = n_components
        self.window = window
        self.model = GaussianHMM(
            n_components=n_components, 
            covariance_type="full", 
            n_iter=100, 
            random_state=42
        )
        
    def fit_predict(self, df):
        """Fit HMM and predict regimes"""
        features = df[['returns', 'volatility', 'trend']].iloc[self.window:].values
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        
        self.model.fit(features_scaled)
        states = self.model.predict(features_scaled)
        
        full_states = np.concatenate([np.zeros(self.window) - 1, states])
        return full_states

print("RegimeDetector class defined!")

In [ ]:
# Visualize HMM Regimes
detector = RegimeDetector(n_components=3, window=30)
regimes = detector.fit_predict(df)

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

colors = ['green', 'yellow', 'red']
for regime in range(3):
    mask = regimes == regime
    axes[0].scatter(df['time'][mask], df['close'][mask], 
                   c=colors[regime], label=f'Regime {regime}', alpha=0.6, s=10)

axes[0].set_title('FPT Close Price by Market Regime (HMM)', fontsize=14)
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['time'], regimes, drawstyle='steps', color='blue', linewidth=1)
axes[1].set_title('Regime Timeline', fontsize=14)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Regime')
axes[1].set_yticks([0, 1, 2])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRegime Distribution:")
for r in range(3):
    count = (regimes == r).sum()
    pct = count / len(regimes) * 100
    print(f"  Regime {r}: {count} days ({pct:.1f}%)")

In [ ]:
# Visualize HMM Regimes (4 States)
detector = RegimeDetector(n_components=4, window=30)
regimes = detector.fit_predict(df)

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# 4 màu cho 4 regimes
colors = ['green', 'yellow', 'orange', 'red'] 

for regime in range(4):
    mask = regimes == regime
    # Dùng try-except đề phòng trường hợp HMM ra ít hơn 4 nhóm (hiếm gặp)
    if mask.sum() > 0:
        axes[0].scatter(df['time'][mask], df['close'][mask], 
                       c=colors[regime], label=f'Regime {regime}', alpha=0.6, s=10)

axes[0].set_title('FPT Close Price by Market Regime (HMM - 4 States)', fontsize=14)
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(df['time'], regimes, drawstyle='steps', color='blue', linewidth=1)
axes[1].set_title('Regime Timeline', fontsize=14)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Regime')

axes[1].set_yticks([0, 1, 2, 3]) 
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRegime Distribution:")
for r in range(4):
    count = (regimes == r).sum()
    pct = count / len(regimes) * 100
    print(f"  Regime {r}: {count} days ({pct:.1f}%)")

## 7. RevIN (Reversible Instance Normalization)

In [ ]:
class RevIN(nn.Module):
    """
    Reversible Instance Normalization.
    Handles distribution shift in time series forecasting.
    """
    def __init__(self, num_features: int, eps=1e-5, affine=True):
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        if self.affine:
            self._init_params()

    def _init_params(self):
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim-1))
        self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps * self.eps)
        x = x * self.stdev
        x = x + self.mean
        return x

    def forward(self, x, mode: str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        return x

print("RevIN class defined!")

## 8. LTSF-Linear Models (All Variants)

In [ ]:
# =====================================================
# GLINEAR MODEL
# =====================================================

class GLinear(nn.Module):
    """
    GLinear: Gaussian-activated Linear Model
    Structure: RevIN -> Permute -> Linear -> GELU -> Linear -> Permute -> RevIN
    """
    def __init__(self, seq_len, pred_len, num_features):
        super(GLinear, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_features = num_features
        
        self.Linear = nn.Linear(self.seq_len, self.seq_len)
        self.GeLU = nn.GELU()
        self.Hidden1 = nn.Linear(self.seq_len, self.pred_len)
        
        self.revin_layer = RevIN(self.num_features)

    def forward(self, x):
        # x shape: [Batch, Seq_Len, Channels]
        x = self.revin_layer(x, 'norm')
        x = x.permute(0, 2, 1) # [Batch, Channels, Seq_Len]
        
        x = self.Linear(x)
        x = self.GeLU(x)
        x = self.Hidden1(x)
        
        x = x.permute(0, 2, 1) # [Batch, Pred_Len, Channels]
        x = self.revin_layer(x, 'denorm')
        return x

class Uni_GLinear(nn.Module):
    """Univariate GLinear Wrapper"""
    def __init__(self, seq_len, pred_len, num_features):
        super().__init__()
        # For univariate, num_features is 1
        self.model = GLinear(seq_len, pred_len, 1)
        
    def forward(self, x):
        # x: [Batch, Seq, 1]
        x_in = x[:, :, 0:1]
        out = self.model(x_in)
        return out[:, :, 0] # Return [Batch, Pred]

class Multi_GLinear(nn.Module):
    """Multivariate GLinear Wrapper"""
    def __init__(self, seq_len, pred_len, num_features):
        super().__init__()
        self.model = GLinear(seq_len, pred_len, num_features)
        
    def forward(self, x):
        # x: [Batch, Seq, Features]
        out = self.model(x)
        # We only care about the target column (index 0) for loss/eval
        return out[:, :, 0]

print("GLinear model defined (Univariate & Multivariate)")

In [ ]:
# Merged into GLinear definitions above

In [ ]:
# =====================================================
# MODEL FACTORY
# =====================================================

MODEL_REGISTRY = {
    ('Univariate', 'GLinear'): Uni_GLinear,
    ('Multivariate', 'GLinear'): Multi_GLinear,
}

def create_model(variant, model_type, seq_len, pred_len, num_features):
    """Factory function to create model"""
    key = (variant, model_type)
    if key not in MODEL_REGISTRY:
        raise ValueError(f"Unknown model configuration: {key}")
    return MODEL_REGISTRY[key](seq_len, pred_len, num_features)

print(f"\nTotal model configurations: {len(MODEL_REGISTRY)}")
for key in MODEL_REGISTRY:
    print(f"  - {key[0]}_{key[1]}")

## 9. Trainer Class

In [ ]:
class Trainer:
    """Training manager with early stopping and learning rate scheduling."""
    def __init__(self, model, criterion, optimizer, scheduler, patience=10):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.patience = patience
        self.best_loss = float('inf')
        self.best_state = None
        self.counter = 0
        
    def fit(self, train_loader, val_loader, epochs, verbose=False):
        """Train the model with early stopping"""
        for epoch in range(epochs):
            self.model.train()
            train_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                self.optimizer.zero_grad()
                output = self.model(X_batch)
                loss = self.criterion(output, y_batch)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
            
            val_loss = self.evaluate(val_loader)
            self.scheduler.step(val_loss)
            
            if val_loss < self.best_loss:
                self.best_loss = val_loss
                self.best_state = deepcopy(self.model.state_dict())
                self.counter = 0
            else:
                self.counter += 1
                
            if self.counter >= self.patience:
                break
                
        if self.best_state:
            self.model.load_state_dict(self.best_state)
            
    def evaluate(self, loader):
        """Evaluate model on a dataset"""
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                output = self.model(X_batch)
                loss = self.criterion(output, y_batch)
                total_loss += loss.item()
        return total_loss / len(loader)

print("Trainer class defined!")

## 10. Grid Search Pipeline

In [ ]:
def inverse_transform(log_data):
    """Convert log-transformed data back to original scale"""
    return np.expm1(log_data)


def train_model_func(variant, model_type, seq_len, num_features, X_train, y_train, epochs=EPOCHS):
    """Helper function to train a single model"""
    train_loader = DataLoader(
        TimeSeriesDataset(X_train, y_train), 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=0, 
        pin_memory=True
    )
    
    model = create_model(variant, model_type, seq_len, PRED_LEN, num_features).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    trainer = Trainer(model, criterion, optimizer, scheduler, patience=PATIENCE)
    trainer.fit(train_loader, train_loader, epochs)
    
    return model


def evaluate_model_func(model, X, y):
    """Evaluate model and compute MSE on original price scale"""
    model.eval()
    loader = DataLoader(TimeSeriesDataset(X, y), batch_size=BATCH_SIZE, shuffle=False)
    
    preds_log, trues_log = [], []
    with torch.no_grad():
        for X_b, y_b in loader:
            X_b = X_b.to(device)
            out = model(X_b)
            preds_log.append(out.cpu().numpy())
            trues_log.append(y_b.numpy())
    
    preds_log = np.concatenate(preds_log)
    trues_log = np.concatenate(trues_log)
    
    preds_price = inverse_transform(preds_log)
    trues_price = inverse_transform(trues_log)
    
    return mean_squared_error(trues_price.flatten(), preds_price.flatten())


def save_submission(predictions, filename):
    """Save predictions to CSV file"""
    sub_df = pd.DataFrame({
        'id': range(1, PRED_LEN + 1), 
        'close': predictions
    })
    filepath = os.path.join(SUBMISSION_DIR, filename)
    sub_df.to_csv(filepath, index=False)
    return filepath


def save_plot(history_prices, predictions, filename, title):
    """Save forecast plot"""
    # fig, ax = plt.subplots(figsize=(12, 6))
    # ... (Keep commented out or implement if needed)
    pass

print("Training utilities defined!")

In [ ]:
def run_grid_search():
    """
    Main Grid Search Pipeline.
    Iterates over all model configurations.
    """
    results = []
    
    # Feature columns
    feature_cols = ['close_log', 'volume_log', 'HL_Spread', 'OC_Spread']
    target_col_idx = df.columns.get_loc('close_log')
    feature_cols_idx = [df.columns.get_loc(c) for c in feature_cols]
    data_values = df.values
    
    # Calculate total combinations
    total = 0
    for variant in VARIANTS:
        for model_type in MODEL_TYPES:
            for seq_len in SEQ_LENS:
                for use_hmm in USE_HMM_OPTIONS:
                    # Logic bỏ qua HMM cho seq dài
                    if seq_len >= 120 and use_hmm:
                        continue
                    
                    # Cộng số lượng experiment
                    if use_hmm:
                        total += len(N_REGIMES_OPTIONS) * len(REGIME_WINDOWS)
                    else:
                        total += 1
                            
    pbar = tqdm(total=total, desc="Grid Search")
    
    for variant in VARIANTS:
        for model_type in MODEL_TYPES:
            for seq_len in SEQ_LENS:
                for use_hmm in USE_HMM_OPTIONS:
                    
                    # Skip HMM for long sequences
                    if seq_len >= 120 and use_hmm:
                        continue
                    
                    hmm_configs = [(None, None)]
                    if use_hmm:
                        hmm_configs = [(n, w) for n in N_REGIMES_OPTIONS for w in REGIME_WINDOWS]
                    
                    for n_regimes, regime_window in hmm_configs:
                        try:
                            seed_everything(42)
                            
                            # === Data Preparation ===
                            regimes = None
                            if use_hmm:
                                detector = RegimeDetector(n_components=n_regimes, window=regime_window)
                                regimes = detector.fit_predict(df)
                            
                            # Train/Val split (80/20)
                            train_size = int(len(data_values) * 0.8)
                            train_data = data_values[:train_size]
                            val_data = data_values[train_size - seq_len:]
                            
                            X_train, y_train = create_sliding_window(train_data, seq_len, PRED_LEN, target_col_idx, feature_cols_idx)
                            X_val, y_val = create_sliding_window(val_data, seq_len, PRED_LEN, target_col_idx, feature_cols_idx)
                            
                            num_features = len(feature_cols_idx)
                            
                            # === Train Global Model (Baseline) ===
                            global_model = train_model_func(variant, model_type, seq_len, num_features, X_train, y_train)
                            
                            # === Evaluation ===
                            if not use_hmm:
                                val_mse = evaluate_model_func(global_model, X_val, y_val)
                            else:
                                val_mse = evaluate_with_hmm(
                                    global_model, variant, model_type,
                                    seq_len, num_features,
                                    X_train, y_train, X_val, y_val,
                                    regimes, train_size
                                )
                            
                            # === Production Forecast ===
                            # 1. Train Global Model trên full data (hoặc 95% data)
                            prod_train_size = int(len(data_values) * 0.95)
                            prod_train_data = data_values[:prod_train_size]
                            X_prod, y_prod = create_sliding_window(prod_train_data, seq_len, PRED_LEN, target_col_idx, feature_cols_idx)
                            
                            final_model = train_model_func(variant, model_type, seq_len, num_features, X_prod, y_prod, epochs=EPOCHS//2)
                            
                            # Prepare input for forecast
                            last_sequence = data_values[-seq_len:, feature_cols_idx]
                            last_seq_tensor = torch.tensor(last_sequence.astype(np.float32)).unsqueeze(0).to(device)
                            
                            # 2. Base Forecast (Global)
                            final_model.eval()
                            with torch.no_grad():
                                pred_log = final_model(last_seq_tensor).cpu().numpy().flatten()
                            
                            # 3. Forecast Correction with HMM
                            if use_hmm:
                                current_regime = regimes[-1] # Lấy regime của ngày cuối cùng
                                
                                prod_regime_indices = []
                                for i in range(len(X_prod)):
                                    r_idx = i + seq_len - 1
                                    if r_idx < len(regimes):
                                        prod_regime_indices.append(regimes[r_idx])
                                    else:
                                        prod_regime_indices.append(-1)
                                
                                prod_regime_indices = np.array(prod_regime_indices)
                                mask = (prod_regime_indices == current_regime)
                                
                                if mask.sum() > 30:
                                    X_regime = X_prod[mask]
                                    y_regime = y_prod[mask]
                                    
                                    regime_model = train_model_func(variant, model_type, seq_len, num_features, 
                                                                  X_regime, y_regime, epochs=EPOCHS//2)
                                    
                                    regime_model.eval()
                                    with torch.no_grad():
                                        pred_log = regime_model(last_seq_tensor).cpu().numpy().flatten()
                            
                            # === Save Results ===
                            pred_price = inverse_transform(pred_log)
                            
                            hmm_status = f"HMM{n_regimes}W{regime_window}" if use_hmm else "NoHMM"
                            filename = f"Sub_{variant}_{model_type}_{hmm_status}_Seq{seq_len}_MSE{val_mse:.0f}.csv"
                            
                            save_submission(pred_price, filename)
                            save_plot(df['close'], pred_price, filename, 
                                     f"{variant} {model_type} | Seq={seq_len} | MSE={val_mse:.0f}")
                            
                            results.append({
                                'Variant': variant,
                                'Model': model_type,
                                'HMM': hmm_status,
                                'SeqLen': seq_len,
                                'ValMSE': val_mse,
                                'File': filename
                            })
                            
                            pbar.set_postfix({'Last': f"{variant[:3]}_{model_type}_{seq_len}", 'MSE': f"{val_mse:.0f}"})
                            
                        except Exception as e:
                            print(f"\n✗ Error: {variant}_{model_type}_Seq{seq_len}: {e}")
                            import traceback
                            traceback.print_exc()
                        
                        pbar.update(1)
    
    pbar.close()
    return results


def evaluate_with_hmm(global_model, variant, model_type, seq_len, num_features,
                      X_train, y_train, X_val, y_val, regimes, train_size):
    """Evaluate with regime-switching models"""
    regime_models = {}
    train_regimes = regimes[:train_size]
    unique_regimes = np.unique(train_regimes)
    
    # Map index X_train sang Regime
    train_regime_indices = []
    for i in range(len(X_train)):
        r_idx = i + seq_len - 1
        if r_idx < len(train_regimes):
            train_regime_indices.append(train_regimes[r_idx])
        else:
            train_regime_indices.append(-1)
    train_regime_indices = np.array(train_regime_indices)
    
    # Train model riêng cho từng regime
    for r in unique_regimes:
        if r == -1:
            continue
        mask = (train_regime_indices == r)
        if mask.sum() > 30:
            X_r = X_train[mask]
            y_r = y_train[mask]
            regime_models[r] = train_model_func(variant, model_type, seq_len, num_features, X_r, y_r)
    
    val_preds_log, val_trues_log = [], []
    
    global_model.eval()
    for model in regime_models.values():
        model.eval()
    
    with torch.no_grad():
        for i in range(len(X_val)):
            # Xác định regime của mẫu validation hiện tại
            global_idx = train_size + i - 1
            curr_regime = regimes[global_idx] if global_idx < len(regimes) else -1
            
            # Chọn model: Nếu có model regime thì dùng, ko thì dùng global
            selected_model = regime_models.get(curr_regime, global_model)
            
            inp = torch.tensor(X_val[i]).unsqueeze(0).to(device)
            pred = selected_model(inp).cpu().numpy()
            
            val_preds_log.append(pred)
            val_trues_log.append(y_val[i])
    
    val_preds_log = np.concatenate(val_preds_log)
    val_trues_log = np.array(val_trues_log)
    
    pred_price = inverse_transform(val_preds_log)
    true_price = inverse_transform(val_trues_log)
    
    return mean_squared_error(true_price.flatten(), pred_price.flatten())

print("Grid Search pipeline defined!")

## 11. Run Grid Search

In [ ]:
def plot_data_split():
    # 1. Lấy dữ liệu gốc
    df_plot = pd.read_csv('data/FPT_train.csv') # Đảm bảo đường dẫn đúng
    data = df_plot['close'].values
    total_len = len(data)
    
    # 2. Tính toán các điểm cắt (Theo logic của code tối ưu)
    TEST_LEN = 100
    
    # Phần dành cho phát triển model (Dev Set)
    dev_len = total_len - TEST_LEN
    
    # Trong Dev Set, chia 80% Train, 20% Val
    train_len = int(dev_len * 0.8)
    val_len = dev_len - train_len
    
    # 3. Vẽ biểu đồ
    plt.figure(figsize=(15, 6))
    
    # Vẽ vùng Train
    plt.plot(range(0, train_len), data[0:train_len], 
             color='#1f77b4', label=f'TRAIN Set ({train_len} days)')
    
    # Vẽ vùng Validation (nối tiếp Train)
    plt.plot(range(train_len - 1, dev_len), data[train_len - 1:dev_len], 
             color='#ff7f0e', label=f'VALIDATION Set ({val_len} days)')
    
    # Vẽ vùng Test (nối tiếp Val)
    plt.plot(range(dev_len - 1, total_len), data[dev_len - 1:total_len], 
             color='#2ca02c', linewidth=2, label=f'INTERNAL TEST Set ({TEST_LEN} days)')
    
    # Tô màu nền để dễ phân biệt
    plt.axvspan(0, train_len, color='#1f77b4', alpha=0.1)
    plt.axvspan(train_len, dev_len, color='#ff7f0e', alpha=0.1)
    plt.axvspan(dev_len, total_len, color='#2ca02c', alpha=0.1)
    
    # Chú thích
    plt.title(f"Data Splitting Strategy (Total: {total_len} days)", fontsize=14)
    plt.xlabel("Time Steps (Days)")
    plt.ylabel("Close Price")
    plt.legend(loc='upper left', frameon=True, shadow=True)
    plt.grid(True, alpha=0.3)
    
    # In ra index cụ thể
    print(f"{'Set':<15} | {'Start Index':<12} | {'End Index':<12} | {'Length':<10}")
    print("-" * 55)
    print(f"{'TRAIN':<15} | {0:<12} | {train_len:<12} | {train_len}")
    print(f"{'VALIDATION':<15} | {train_len:<12} | {dev_len:<12} | {val_len}")
    print(f"{'INTERNAL TEST':<15} | {dev_len:<12} | {total_len:<12} | {TEST_LEN}")
    print("-" * 55)
    
    plt.show()

plot_data_split()

In [ ]:
print("="*60)
print("STARTING GRID SEARCH")
print("="*60)
print(f"\nGrid Search Space:")
print(f"  - Variants: {VARIANTS}")
print(f"  - Models: {MODEL_TYPES}")
# print(f"  - RevIN: {USE_REVIN_OPTIONS}")
print(f"  - Seq Lengths: {SEQ_LENS}")
print(f"  - HMM: {USE_HMM_OPTIONS}")
print("\n")

results = run_grid_search()

print("\n" + "="*60)
print("GRID SEARCH COMPLETED")
print("="*60)

## 12. Results Analysis

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results).sort_values('ValMSE')
results_df.to_csv('results/grid_search_results.csv', index=False)

print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)

print(f"\nTotal experiments: {len(results_df)}")
print(f"\nTop 15 Best Models:")
display(results_df.head(15))

print(f"\nWorst 5 Models:")
display(results_df.tail(5))

In [ ]:
# Detailed Analysis
print("\n" + "="*60)
print("DETAILED ANALYSIS")
print("="*60)

# By Variant
print("\n📊 Average MSE by Variant:")
print(results_df.groupby('Variant')['ValMSE'].agg(['mean', 'std', 'min', 'max']).round(2))

# By Model Type
print("\n📊 Average MSE by Model Type:")
print(results_df.groupby('Model')['ValMSE'].agg(['mean', 'std', 'min', 'max']).round(2))

# By Sequence Length
print("\n📊 Average MSE by Sequence Length:")
print(results_df.groupby('SeqLen')['ValMSE'].agg(['mean', 'std', 'min', 'max']).round(2))

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Grid Search Results Analysis', fontsize=16, fontweight='bold')

# 1. MSE by Variant
variant_mse = results_df.groupby('Variant')['ValMSE'].mean()
axes[0, 0].bar(variant_mse.index, variant_mse.values, color=['blue', 'green'])
axes[0, 0].set_title('Average MSE by Variant', fontsize=12)
axes[0, 0].set_ylabel('MSE')

# 2. MSE by Sequence Length
seq_mse = results_df.groupby('SeqLen')['ValMSE'].mean()
axes[0, 1].bar(seq_mse.index.astype(str), seq_mse.values, color='purple')
axes[0, 1].set_title('Average MSE by Sequence Length', fontsize=12)
axes[0, 1].set_xlabel('Sequence Length')
axes[0, 1].set_ylabel('MSE')

# 3. MSE by HMM
hmm_mse = results_df.groupby('HMM')['ValMSE'].mean().sort_values()
axes[1, 0].barh(hmm_mse.index, hmm_mse.values, color='orange')
axes[1, 0].set_title('Average MSE by HMM Configuration', fontsize=12)
axes[1, 0].set_xlabel('MSE')

# 4. Top 15 Models
top15 = results_df.head(15)
labels = [f"{r['Variant'][:3]}_{r['Model']}\nSeq={r['SeqLen']}" for _, r in top15.iterrows()]
axes[1, 1].barh(range(len(top15)), top15['ValMSE'].values, color='green')
axes[1, 1].set_yticks(range(len(top15)))
axes[1, 1].set_yticklabels(labels, fontsize=8)
axes[1, 1].set_title('Top 15 Best Models by MSE', fontsize=12)
axes[1, 1].set_xlabel('MSE')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.savefig('results/grid_search_analysis.png', dpi=150)
plt.show()

In [ ]:
# Heatmap: Variant vs Sequence Length
pivot_seq = results_df.pivot_table(
    values='ValMSE', index='Variant', columns='SeqLen', aggfunc='mean'
)
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_seq, annot=True, fmt='.0f', cmap='RdYlGn_r')
plt.title('Average MSE: Variant vs Sequence Length', fontsize=12)
plt.tight_layout()
plt.savefig('results/heatmap_analysis.png', dpi=150)
plt.show()

## 13. Summary và Kết luận

In [ ]:
print("="*60)
print("FINAL SUMMARY")
print("="*60)

print(f"\n📊 Dataset: FPT Stock ({len(df)} days)")
print(f"📅 Date Range: {df['time'].min().date()} to {df['time'].max().date()}")

print(f"\n🔬 Total Experiments: {len(results_df)}")

print(f"\n🏆 TOP 5 BEST MODELS:")
for i, (_, row) in enumerate(results_df.head(5).iterrows(), 1):
    print(f"   {i}. {row['Variant']}_{row['Model']} | Seq={row['SeqLen']} | {row['HMM']} | MSE={row['ValMSE']:.2f}")

print(f"\n📈 KEY FINDINGS:")
best_variant = results_df.groupby('Variant')['ValMSE'].mean().idxmin()
best_model = results_df.groupby('Model')['ValMSE'].mean().idxmin()
best_seq = results_df.groupby('SeqLen')['ValMSE'].mean().idxmin()

print(f"   - Best Variant: {best_variant}")
print(f"   - Best Model Type: {best_model}")
print(f"   - Best Sequence Length: {best_seq}")

print(f"\n📁 OUTPUT FILES:")
print(f"   - Results CSV: results/grid_search_results.csv")
print(f"   - Analysis Plot: results/grid_search_analysis.png")
print(f"   - Heatmap: results/heatmap_analysis.png")
print(f"   - Submissions: {SUBMISSION_DIR}/ ({len(results_df)} files)")

print("\n" + "="*60)
print("DONE! 🎉")
print("="*60)

In [ ]:
# List all submission files
print("\n📂 All Submission Files:")
print("-" * 80)
for i, filename in enumerate(sorted(results_df['File'].tolist()), 1):
    print(f"{i:3d}. {filename}")